In [7]:
import xarray as xr
import matplotlib.pyplot as plt

In [2]:
ds = xr.open_dataset('/data/weather-benchmark/tigge/z500.nc')

In [3]:
ds

<xarray.Dataset>
Dimensions:  (lat: 65, lev: 1, lon: 128, time: 126)
Coordinates:
  * lon      (lon) float64 0.0 2.812 5.625 8.438 ... 348.8 351.6 354.4 357.2
  * lat      (lat) float64 90.0 87.19 84.38 81.56 ... -81.56 -84.38 -87.19 -90.0
  * lev      (lev) float64 5e+04
  * time     (time) datetime64[ns] 2016-01-01 ... 2016-01-08T12:00:00
Data variables:
    gh       (time, lev, lat, lon) float32 ...
Attributes:
    CDI:          Climate Data Interface version ?? (http://mpimet.mpg.de/cdi)
    Conventions:  CF-1.4
    history:      Wed Oct 23 12:27:33 2019: cdo -f nc copy /data/weather-benc...
    institution:  European Centre for Medium-Range Weather Forecasts
    CDO:          Climate Data Operators version 1.7.0 (http://mpimet.mpg.de/...

In [12]:
ds.time

<xarray.DataArray 'time' (time: 126)>
array(['2016-01-01T00:00:00.000000000', '2016-01-01T06:00:00.000000000',
       '2016-01-01T12:00:00.000000000', '2016-01-01T18:00:00.000000000',
       '2016-01-02T00:00:00.000000000', '2016-01-02T06:00:00.000000000',
       '2016-01-02T12:00:00.000000000', '2016-01-02T18:00:00.000000000',
       '2016-01-03T00:00:00.000000000', '2016-01-03T06:00:00.000000000',
       '2016-01-03T12:00:00.000000000', '2016-01-03T18:00:00.000000000',
       '2016-01-04T00:00:00.000000000', '2016-01-04T06:00:00.000000000',
       '2016-01-04T12:00:00.000000000', '2016-01-04T18:00:00.000000000',
       '2016-01-05T00:00:00.000000000', '2016-01-05T06:00:00.000000000',
       '2016-01-05T12:00:00.000000000', '2016-01-05T18:00:00.000000000',
       '2016-01-06T00:00:00.000000000', '2016-01-01T12:00:00.000000000',
       '2016-01-01T18:00:00.000000000', '2016-01-02T00:00:00.000000000',
       '2016-01-02T06:00:00.000000000', '2016-01-02T12:00:00.000000000',
       '2016-